In [1]:
import requests
import re
import json
# 错误信息
from requests.exceptions import RequestException
# 进程池
from multiprocessing import Pool

In [2]:
# 页面信息
def get_page(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        return None
    except RequestException:
        return None
# 解析页面提取信息
def parse_page(html):
    # 生成正则对象
    # re.S -> 忽略换行
    pattern = re.compile('<dd>.*?board-index.*?>(\d+)</i>.*?data-src="(.*?)".*?name"><a.*?>(.*?)</a>.*?star">(.*?).*?releasetime">(.*?)</p>.*?integer">(.*?).*?fraction">(.*?)</i>.*?</dd>', re.S)
    items = re.findall(pattern, html)
#     print(items)
    # 数据清洗
    # yield -> 生成器，调用时才会返回值
    for item in items:
        yield {
            'index': item[0],
            'image': item[1],
            'title': item[2],
            'actor': item[3].strip()[3:], # 取下标为 3 的值之后的数据
            'time': item[4].strip()[5:],
            'score': item[5] + item[6]
        }
# 写入文件
def wirte_to_file(content):
    with open('result.txt', 'a', encoding='utf-8') as f:
        # 转为json格式，不转ascii码，换行
        f.write(json.dumps(content, ensure_ascii=False) + '\n')
def main(offset):
    url = 'http://maoyan.com/board/4?offset=' + str(offset)
    html = get_page(url)
#     print(html)
    for item in parse_page(html):
        print(item)
        wirte_to_file(item)
if __name__ == '__main__':
    # 创建进程池
    pool = Pool()
    pool.map(main, [i * 10 for i in range(10)])
    pool.close()
    pool.join()